In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.insert(0, '/content/drive/My Drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np

import torch
from torchvision import datasets
from torchvision import transforms

data_dir = "/drive/My Drive/spiders_new/"
os.chdir("/content/drive/My Drive/spiders2_new_dataset/")
print(os.getcwd())



/content/drive/My Drive/spiders2_new_dataset


In [3]:
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser()
parser.add_argument("--ngpu", type=int, default=1, help='Number of gpu.')
parser.add_argument("--d_conv_dim", type=int, default=128, help='Discriminator convolution dimension.')  # model hyperparameter
parser.add_argument("--g_conv_dim", type=int, default=128, help='Generator convolution dimension.')  # model hyperparameter
parser.add_argument("--z_size", type=int, default=100, help='Noise size - input to image generation.') # model hyperparameter
parser.add_argument("--n_epochs", type=int, default=900, help='Number of epochs of training.')
parser.add_argument("--img_size", type=int, default=128, help='Size of each image dimension.')
parser.add_argument("--batch_size", type=int, default=64, help="Size of the batches.")
parser.add_argument("--real_label", type=float, default=1, help="Label for real images.")
parser.add_argument("--fake_label", type=float, default=0, help="Label for fake images.")
parser.add_argument("--Loss_name: ", type=str, default="BCELoss", help="Show the name of used loss function.")
parser.add_argument("--Optim_name1: ", type=str, default="Adam", help="Show the name of the 1st used optimizator.")
parser.add_argument("--Optim_name2: ", type=str, default="RMSprop", help="Show the name of the 2nd used optimizator.")
parser.add_argument("--Optim_name3: ", type=str, default="SGD", help="Show the name of the 3rd used optimizator.")
parser.add_argument("--lr", type=float, default=0.0002, help="Learning rate")
parser.add_argument("--beta1", type=float, default=0.5, help="First parameter to loss function(if it's necessary)")
parser.add_argument("--beta2", type=float, default=0.999, help="Second parameter to loss function(if it's necessary)")
opt = parser.parse_args()

# To easy save the parameters to the .txt file 
parserString = ""
for arg in vars(opt):
    parserString += (arg + " " + str(getattr(opt, arg)) + "\n")

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and opt.ngpu > 0) else "cpu")

In [4]:
def get_dataloader(batch_size, image_size, data_dir=os.getcwd()):
    """
    Batch the neural network data using DataLoader
    :param batch_size: The size of each batch; the number of images in a batch
    :param img_size: The square size of the image data (x, y)
    :param data_dir: Directory where image data is located
    :return: DataLoader with batched data
    """
    print("hey man")
    print(data_dir)
    dataset = dset.ImageFolder(root=data_dir,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
    # Create the dataloader
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=1)
    return dataloader

In [5]:
# Call your function and get a dataloader
spiders_train_loader = get_dataloader(opt.batch_size, opt.img_size)

hey man
/content/drive/My Drive/spiders2_new_dataset


In [6]:
if (device.type == 'cuda') and (opt.ngpu > 1):
    print("\nThis is True. I'm using cuda and ngpu is bigger than 1.\n")
    print("The type of device is %s. The nb of gpu is %d. \n" %(device.type, opt.ngpu))

# Plot some training images
real_batch = next(iter(spiders_train_loader))
plt.figure(figsize=(30,30))
plt.axis("off")
plt.title("Training Images", color='white')
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [8]:
class Discriminator(nn.Module):
    def __init__(self, conv_dim=64, ngpu=1):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.conv_dim = conv_dim
        self.main = nn.Sequential(
            # input is 3 x 128 x 128
            nn.Conv2d(3, conv_dim, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. conv_dim x 64 x 64
            nn.Conv2d(conv_dim, conv_dim * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (conv_dim*2) x 32 x 32
            nn.Conv2d(conv_dim * 2, conv_dim * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (conv_dim*4) x 16 x 16
            nn.Conv2d(conv_dim * 4, conv_dim * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (conv_dim*8) x 8 x 8
            nn.Conv2d(conv_dim * 8, conv_dim * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (conv_dim*16) x 4 x 4
            nn.Conv2d(conv_dim * 16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [9]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, z_size, conv_dim=64, ngpu=1):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.conv_dim = conv_dim
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( z_size, conv_dim * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(conv_dim * 16),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d( conv_dim * 16, conv_dim * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d(conv_dim * 8, conv_dim * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16
            nn.ConvTranspose2d( conv_dim * 4, conv_dim * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d(conv_dim * 2, conv_dim, 4, 2, 1, bias=False),
            nn.BatchNorm2d(conv_dim),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d(conv_dim, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )

    def forward(self, input):
        return self.main(input)

In [10]:
# Create the dictionary of different optimizers with different values
optimizers = {}

# Initialize BCELoss function
criterion = nn.BCELoss()

def gan_init(number):
    # ==============================
    # CREATE THE DISCRIMINATOR
    # ==============================
    netD = Discriminator(opt.d_conv_dim, opt.ngpu).to(device)

    # Handle multi-gpu if desired
    if (device.type == 'cuda') and (opt.ngpu > 1):
        netD = nn.DataParallel(netD, list(range(opt.ngpu)))

    # Apply the weights_init function to randomly initialize all weights
    #  to mean=0, stdev=0.2.
    netD.apply(weights_init)

    # ==============================
    # CREATE THE GENERATOR
    # ==============================
    netG = Generator(opt.z_size, opt.g_conv_dim, opt.ngpu).to(device)

    # Handle multi-gpu if desired
    if (device.type == 'cuda') and (opt.ngpu > 1):
        netG = nn.DataParallel(netG, list(range(opt.ngpu)))

    # Apply the weights_init function to randomly initialize all weights
    #  to mean=0, stdev=0.2.
    netG.apply(weights_init)

    if (number == 1):
        optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, opt.beta2))
        optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, opt.beta2))
        optimizers['Adam'] = [optimizerD, optimizerG]
    elif (number == 2):
        optimizerD = optim.RMSprop(netD.parameters(), lr=opt.lr, alpha=opt.beta1)
        optimizerG = optim.RMSprop(netG.parameters(), lr=opt.lr, alpha=opt.beta1)
        optimizers['RMSprop'] = [optimizerD, optimizerG]
    elif (number == 3):
        optimizerD = optim.SGD(netD.parameters(), lr=opt.lr)
        optimizerG = optim.SGD(netG.parameters(), lr=opt.lr)
        optimizers['SGD'] = [optimizerD, optimizerG]
    
    return netD, netG, optimizerD, optimizerG

In [ ]:
## Training Loop

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

# Arrays where the main info will be stored
img_list_all = []
G_losses_epochs_all = []
D_losses_epochs_all = []
D_x_losses_epochs_all = []
D_G_z1_losses_epochs_all = []
D_G_z2_losses_epochs_all = []
output_train_stat_all = []

for optim_number in range (3,4):
    if (optim_number == 1):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/Adam")
    elif (optim_number == 2):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/RMSprop")
    elif (optim_number == 3):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/SGD")

    netD, netG, optimizerD, optimizerG = gan_init(optim_number)
    
    # Lists to keep track of progress
    img_list = []
    G_losses_epochs = []
    D_losses_epochs = []
    D_x_losses_epochs = []
    D_G_z1_losses_epochs = []
    D_G_z2_losses_epochs = []
    output_train_stat = []

    print("Starting Training Loop...")
    # For each epoch
    for epoch in range(opt.n_epochs):
        D_x_losses = []
        D_G_z1_losses = []
        D_G_z2_losses = []        
        G_losses = []
        D_losses = []
        # For each batch in the dataloader
        for i, data in enumerate(spiders_train_loader, 0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), opt.real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, 100, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label.fill_(opt.fake_label)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Add the gradients from the all-real and all-fake batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(opt.real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()

            # Output training stats
            if i % 50 == 0:
                train_stat = '[%d/%d][%d/%d]\tOpt_numb: %d\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'  % (epoch, opt.n_epochs, i, len(spiders_train_loader), optim_number, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2)
                output_train_stat.append(train_stat + "\n") 
                print(train_stat)


            D_x_losses.append(D_x)
            D_G_z1_losses.append(D_G_z1)
            D_G_z2_losses.append(D_G_z2)
            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())

            # Check how the generator is doing by saving G's output on fixed_noise
            # It saves an image: 1) every 50th epochs; 2) the last epoch
            if ((epoch % 50 == 0) and (i == len(spiders_train_loader)-1)) or ((epoch == opt.n_epochs-1) and (i == len(spiders_train_loader)-1)):
                with torch.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
                
                f_name_image = "Im_%dep_%dbs_%dis" % ((epoch+1), opt.batch_size, opt.img_size)
                # Plot the fake images from the last epoch
                fig = plt.figure(figsize=(40,40))
                #plt.subplot(1,2,2)
                plt.axis("off")
                plt.title("Fake Images", color='black')
                plt.imshow(np.transpose(img_list[-1],(1,2,0)))
                #fig.savefig('Images/v1_11/v1_11-' + f_name_image + '.png')
                fig.savefig('v1_13-' + f_name_image + '.png')
                plt.close(fig)

        D_x_losses_aver_per_epoch = round((sum(D_x_losses) / len(D_x_losses)),5)
        D_G_z1_losses_aver_per_epoch = round((sum(D_G_z1_losses) / len(D_G_z1_losses)),5)
        D_G_z2_losses_aver_per_epoch = round((sum(D_G_z2_losses) / len(D_G_z2_losses)),5)
        G_losses_aver_per_epoch = round((sum(G_losses) / len(G_losses)),5)
        D_losses_aver_per_epoch = round((sum(D_losses) / len(D_losses)),5)

        D_x_losses_epochs.append(D_x_losses_aver_per_epoch)
        D_G_z1_losses_epochs.append(D_G_z1_losses_aver_per_epoch)
        D_G_z2_losses_epochs.append(D_G_z2_losses_aver_per_epoch)
        G_losses_epochs.append(G_losses_aver_per_epoch)
        D_losses_epochs.append(D_losses_aver_per_epoch)

    img_list_all.append(img_list)
    G_losses_epochs_all.append(G_losses_epochs)
    D_losses_epochs_all.append(D_losses_epochs)
    D_x_losses_epochs_all.append(D_x_losses_epochs)
    D_G_z1_losses_epochs_all.append(D_G_z1_losses_epochs)
    D_G_z2_losses_epochs_all.append(D_G_z2_losses_epochs)
    output_train_stat_all.append(output_train_stat)

    #SAVE ALL NECESSARY INFO 
    if (optim_number == 1):
        os.chdir("/content/drive/My Drive/results/Structures/v1_13/Adam")
    elif (optim_number == 2):
        os.chdir("/content/drive/My Drive/results/Structures/v1_13/RMSprop")
    elif (optim_number == 3):
        os.chdir("/content/drive/My Drive/results/Structures/v1_13/SGD")


    wrOutput = str(output_train_stat_all[-1])
    with open('v1_13-output.txt', 'w') as wrFileOutput:
        wrFileOutput.write(wrOutput)

    # SAVE THE GENERATOR INFO
    wrOptG = str(optimizerG)
    with open('v1_13-optimizerG.txt', 'w') as wrFileOptG:
        wrFileOptG.write(wrOptG)

    torch.save(netG, 'v1_13-netG.pth')  # save entire net G
    torch.save(netG.state_dict(), 'v1_13-netG_params.pth')  # save parameters of generator (weigths, gradients)
    torch.save(optimizerG, 'v1_13-optimizerG.pth')  # save the generator optimizer
    torch.save(optimizerG.state_dict(), 'v1_13-optimizerG_params.pth')  # save the parameters of the generator optimizer

    with open('v1_13-G_Loss_ep.txt', 'w') as wrGLoss:
        for valEp in G_losses_epochs_all[-1]:
            wrGLoss.write("%s\n" % valEp)

    with open('v1_13-DGz2_Loss_ep.txt', 'w') as wrDGz2Loss:
        for valEp in D_G_z2_losses_epochs_all[-1]:
            wrDGz2Loss.write("%s\n" % valEp)


    # SAVE THE DISCRIMINATOR INFO
    wrOptD = str(optimizerD)
    with open('v1_13-optimizerD.txt', 'w') as wrFileOptD:
        wrFileOptD.write(wrOptD)

    torch.save(netD, 'v1_13-netD.pth')  # save entire net D
    torch.save(netD.state_dict(), 'v1_13-netD_params.pth')  # save parameters of discriminator (weigths, gradients)
    torch.save(optimizerD, 'v1_13-optimizerD.pth')  # save the discriminator optimizer
    torch.save(optimizerD.state_dict(), 'v1_13-optimizerD_params.pth')  # save the parameters of the discriminator optimizer          

    with open('v1_13-D_Loss_ep.txt', 'w') as wrDLoss:
        for valEp in D_losses_epochs_all[-1]:
            wrDLoss.write("%s\n" % valEp)

    with open('v1_13-Dx_Loss_ep.txt', 'w') as wrDxLoss:
        for valEp in D_x_losses_epochs_all[-1]:
            wrDxLoss.write("%s\n" % valEp)

    with open('v1_13-DGz1_Loss_ep.txt', 'w') as wrDGz1Loss:
        for valEp in D_G_z1_losses_epochs_all[-1]:
            wrDGz1Loss.write("%s\n" % valEp)


Starting Training Loop...
[0/900][0/342]	Opt_numb: 3	Loss_D: 2.3762	Loss_G: 2.6495	D(x): 0.2707	D(G(z)): 0.1709 / 0.1934
[0/900][50/342]	Opt_numb: 3	Loss_D: 0.8182	Loss_G: 2.1383	D(x): 0.8079	D(G(z)): 0.3145 / 0.2115
[0/900][100/342]	Opt_numb: 3	Loss_D: 0.7557	Loss_G: 2.6470	D(x): 0.8995	D(G(z)): 0.3208 / 0.2072
[0/900][150/342]	Opt_numb: 3	Loss_D: 0.9315	Loss_G: 2.5960	D(x): 0.7981	D(G(z)): 0.2807 / 0.1657
[0/900][200/342]	Opt_numb: 3	Loss_D: 1.1057	Loss_G: 2.7673	D(x): 0.7164	D(G(z)): 0.2266 / 0.1505
[0/900][250/342]	Opt_numb: 3	Loss_D: 0.8622	Loss_G: 3.4868	D(x): 0.8109	D(G(z)): 0.1678 / 0.0845
[0/900][300/342]	Opt_numb: 3	Loss_D: 1.1220	Loss_G: 3.5917	D(x): 0.7575	D(G(z)): 0.2104 / 0.1140
[1/900][0/342]	Opt_numb: 3	Loss_D: 0.5444	Loss_G: 3.8190	D(x): 0.8557	D(G(z)): 0.1857 / 0.0756
[1/900][50/342]	Opt_numb: 3	Loss_D: 0.5473	Loss_G: 3.6360	D(x): 0.9305	D(G(z)): 0.2849 / 0.0632
[1/900][100/342]	Opt_numb: 3	Loss_D: 0.8092	Loss_G: 3.4890	D(x): 0.8564	D(G(z)): 0.1585 / 0.0652
[1/900][15

In [ ]:
# # Important

# # os.chdir("/content/drive/My Drive/results")

# torch.save(opt, 'Structures/v1_12/v1_12-parser.pth')

# text_file = open("Structures/v1_12/v1_12-parser_params.txt", "w")
# n = text_file.write(parserString)
# text_file.close()

# wrGStruct = str(netG)
# with open('Structures/v1_12/v1_12-netG_struct.txt', 'w') as wrFileGStruct:
#     wrFileGStruct.write(wrGStruct)

# wrDStruct = str(netD)
# with open('Structures/v1_12/v1_12-netD_struct.txt', 'w') as wrFileDStruct:
#     wrFileDStruct.write(wrDStruct)


In [ ]:
# SAVE THE LOSSES

os.chdir("/content/drive/My Drive/results")

for optim_number in range (3,4):
    if (optim_number == 1):
        name = "Adam"
        g_label="G_Adam"
        d_label="D_Adam"
    elif (optim_number == 2):
        name = "RMS"
        g_label="G_RMSprop"
        d_label="D_RMSprop"
    elif (optim_number == 3):
        name = "SGD"
        g_label="G_SGD"
        d_label="D_SGD"

    f_name_loss = "Loss_%s_%dep_%dbs_%dis" % (name, opt.n_epochs, opt.batch_size, opt.img_size)

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    plt.grid(color='black')
    plt.title("Generator and Discriminator Loss During Training", color='black', size = 20)
    plt.plot(G_losses_epochs_all[-1],label=g_label)
    plt.plot(D_losses_epochs_all[-1],label=d_label)
    plt.xlabel("Epochs", color='black', size = 13)
    plt.ylabel("Loss", color='black', size=13)
    ax.tick_params(axis='x', colors='black')
    ax.tick_params(axis='y', colors='black')
    import matplotlib.ticker as plticker
    loc = plticker.MultipleLocator(base=100)
    locY = plticker.MultipleLocator(base=5)
    ax.xaxis.set_major_locator(loc)
    ax.yaxis.set_major_locator(locY)
    plt.legend()
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '.png')

    plt.ylim(-50,50)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD2.png')
    plt.ylim(0,50)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD3.png')
    plt.ylim(0,100)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD4.png')
    plt.ylim(0,20)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD5.png')
    plt.ylim(-100,100)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD6.png')
    plt.ylim(-10,10)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD7.png')
    plt.ylim(-20,20)
    fig.savefig('Losses/v1_13/v1_13-' + f_name_loss + '_AD8.png')
    plt.close(fig)




In [ ]:
for optim_number in range (3,4):
    if (optim_number == 1):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/Adam")
    elif (optim_number == 2):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/RMSprop")
    elif (optim_number == 3):
        os.chdir("/content/drive/My Drive/results/Images/v1_13/SGD")

    fig = plt.figure(figsize=(30,30))
    plt.axis("off")
    ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list_all[-1]]
    ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

    ani.save('v1_13-Animation_training.mp4')

    HTML(ani.to_jshtml())

In [ ]:


#optim_keys = list(optimizers.keys())


#====================================================================================================================
#                             1 Optimizer: Adam.
#====================================================================================================================
# Setup Adam optimizers for both G and D
# optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, opt.beta2))
# optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, opt.beta2))

#====================================================================================================================
#                             2 Optimizer: ASGD.
#====================================================================================================================
# # Setup ASGD optimizers for both G and D
# optimizerD = optim.ASGD(netD.parameters(), lr=lr)
# optimizerG = optim.ASGD(netG.parameters(), lr=lr)

#====================================================================================================================
#                             3 Optimizer: RMSprop.
#====================================================================================================================
# # Setup RMSprop optimizers for both G and D
# optimizerD = optim.RMSprop(netD.parameters(), lr=lr, alpha=beta1)
# optimizerG = optim.RMSprop(netG.parameters(), lr=lr, alpha=beta1)

#====================================================================================================================
#                             4 Optimizer: SGD.
#====================================================================================================================
# # Setup SGD optimizers for both G and D
# optimizerD = optim.SGD(netD.parameters(), lr=lr)
# optimizerG = optim.SGD(netG.parameters(), lr=lr)

#====================================================================================================================
#                             5 Optimizer: Rprop.
#====================================================================================================================
# # Setup Rprop optimizers for both G and D
# optimizerD = optim.Rprop(netD.parameters(), lr=lr, etas=(beta1, 1.2))
# optimizerG = optim.Rprop(netG.parameters(), lr=lr, etas=(beta1, 1.2))